In [4]:
import subprocess
import os

netlist_template = """

*CONDUCTION NMOS W1

.INCLUDE ../../model/32nm_HP.pm
.OPTIONS GMIN=1e-020 ABSTOL=1e-18

*Definizione dei parametri
.PARAM Lmin=32n
.PARAM Wmin=32n
.PARAM Ldiff=64n

.TEMP 85

.PARAM N=2

*Descrizione della prima cella NMOS
Mn1 out in1 x 0 nmos W={{Wmin*N}} L={{Lmin}} AS={{Wmin*N*Ldiff}} AD={{Wmin*N*Ldiff}} PS={{2*(Ldiff+Wmin*N)}} PD={{2*(Ldiff+Wmin*N)}}

*Descrizione della seconda cella NMOS in series
Mn2 x in2 0 0 nmos W={{Wmin*N}} L={{Lmin}} AS={{Wmin*N*Ldiff}} AD={{Wmin*N*Ldiff}} PS={{2*(Ldiff+Wmin*N)}} PD={{2*(Ldiff+Wmin*N)}}

Vdd out 0 {output}
Va 	in1	0 {inp1}
Vb  in2 0 {inp2}

.CONTROL
dc TEMP 85 86 10
print V(x)

.ENDC
.END

"""

inp1s = [0, 0, 1, 1]
inp2s = [0, 1, 0, 1]

outputs = [1, 1, 1, 0]

output_dir = "diff_inputs"

# make the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

outs = []
i = 0
for inp1, inp2 in zip(inp1s, inp2s):
    netlist_content = netlist_template.format(output=outputs[i], inp1=inp1, inp2=inp2)
    i += 1
    
    filename = f"{output_dir}/nmos_{inp1},{inp2}.net"
    
    with open(filename, 'w') as file:
        file.write(netlist_content)
    # add a -b and -o flag also pipe it to a logs.txt file
    p = subprocess.run(["ngspice", filename] , encoding='utf-8', stdout=subprocess.PIPE)
    windstr = p.stdout
    outs.append(windstr)

    
      

Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run


In [5]:
matrix = []
for o in outs:
    lines = o.split('\n')
    matrix.append(lines[10])
    
print(matrix)

['v(x) = 1.208642e-01', 'v(x) = 5.964721e-05', 'v(x) = 7.457991e-01', 'v(x) = 1.498350e-07']


In [6]:
inp1s = [0, 0, 1, 1]
inp2s = [0, 1, 0, 1]

filename = f"outputs.csv"
with open(filename, 'w') as file:
    file.write("inp1,inp2,V(Internal node)\n")
    for i, line in enumerate(matrix):
        file.write(f"{inp1s[i]},{inp2s[i]},{line.split(' ')[-1]}\n")